In [1]:
from italian_csv_type_prediction.dataframe_generators import SimpleDatasetGenerator
from italian_csv_type_prediction.models import TypePredictor
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
from multiprocessing import Pool, cpu_count
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [2]:
def dataset_generation(number:int):
    return SimpleDatasetGenerator().build(number, verbose=False)

def _dataset_generation(args):
    return dataset_generation(*args)

def parallel_dataset_generation(number:int):
    processes = min(cpu_count()*5, number)
    with Pool(cpu_count()) as p:
        Xs, ys = list(zip(*tqdm(
            p.imap(_dataset_generation, (
                (number//processes, )
                for _ in range(processes)
            )),
            desc="Creating dataset",
            total=processes,
            leave=False
        )))
    return np.vstack(Xs), np.concatenate(ys)

In [3]:
x_train, y_train = parallel_dataset_generation(2000)
x_test, y_test = parallel_dataset_generation(2000)

In [4]:
model = TypePredictor()

model.fit(x_train, y_train)

In [5]:
y_pred = model._model.predict(x_test)
y_train_pred = model._model.predict(x_train)

In [6]:
accuracy_score(y_test, y_pred), balanced_accuracy_score(y_test, y_pred)

(0.9796285838035724, 0.9766219908558527)

In [7]:
accuracy_score(y_train, y_train_pred), balanced_accuracy_score(y_train, y_train_pred)

(0.9982254601250911, 0.9983257923360931)

In [8]:
X, y = SimpleDatasetGenerator().generate_simple_dataframe()

In [9]:
y

,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,CountryCode,Name,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean,NumericId
0,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,NaN,Address,ItalianZIPCode,NaN,Region,NaN,...,NaN,Name,Surname,String,EMail,PhoneNumber,NaN,BiologicalSex,Boolean,NumericId
1,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,Plate,Address,ItalianZIPCode,ProvinceCode,Region,NaN,...,CountryCode,Name,NaN,NaN,NaN,PhoneNumber,Date,NaN,Boolean,NumericId
2,ItalianFiscalCode,ItalianVAT,CadastreCode,NaN,Plate,Address,NaN,ProvinceCode,Region,Municipality,...,CountryCode,Name,Error,String,EMail,PhoneNumber,NaN,BiologicalSex,NaN,NumericId
3,ItalianFiscalCode,ItalianVAT,CadastreCode,NaN,Plate,Address,ItalianZIPCode,ProvinceCode,Error,Municipality,...,CountryCode,NaN,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean,NumericId
4,NaN,ItalianVAT,CadastreCode,Document,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,NaN,NaN,Surname,String,EMail,PhoneNumber,NaN,BiologicalSex,Boolean,NumericId
5,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,Plate,Address,NaN,ProvinceCode,Region,Municipality,...,CountryCode,Name,NaN,NaN,EMail,NaN,Date,BiologicalSex,NaN,NumericId
6,NaN,ItalianVAT,CadastreCode,Document,Plate,Address,ItalianZIPCode,NaN,Region,Municipality,...,CountryCode,NaN,Surname,String,EMail,PhoneNumber,NaN,BiologicalSex,Boolean,NumericId
7,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,CountryCode,Name,NaN,NaN,NaN,PhoneNumber,Date,BiologicalSex,Boolean,NumericId
8,ItalianFiscalCode,NaN,NaN,Document,Plate,Address,NaN,ProvinceCode,Region,NaN,...,CountryCode,Name,Error,String,EMail,Error,Date,NaN,NaN,NumericId
9,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,NaN,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,CountryCode,NaN,NaN,NaN,EMail,PhoneNumber,Date,BiologicalSex,Boolean,NumericId


In [10]:
from collections import Counter

mask = y_test != y_pred

true_labels = model._embedder._encoder.inverse_transform(y_test[mask])
predicted_labels = model._embedder._encoder.inverse_transform(y_pred[mask])

Counter(zip(true_labels, predicted_labels))

Counter({('Error', 'Document'): 323,
         ('Name', 'Region'): 106,
         ('Error', 'Region'): 41,
         ('Surname', 'Error'): 520,
         ('Error', 'NaN'): 12,
         ('Error', 'ProvinceCode'): 92,
         ('Error', 'Integer'): 195,
         ('Country', 'Error'): 534,
         ('Error', 'Country'): 737,
         ('Error', 'Surname'): 386,
         ('Error', 'String'): 1236,
         ('NaN', 'CountryCode'): 182,
         ('Region', 'Country'): 136,
         ('Name', 'Country'): 3067,
         ('EMail', 'Document'): 3,
         ('EMail', 'Error'): 7,
         ('String', 'Error'): 1518,
         ('Error', 'Name'): 518,
         ('Municipality', 'Name'): 21,
         ('Surname', 'Country'): 455,
         ('String', 'Address'): 2431,
         ('Date', 'Error'): 70,
         ('Error', 'PhoneNumber'): 6,
         ('Address', 'String'): 2391,
         ('Document', 'Surname'): 13,
         ('Error', 'Boolean'): 70,
         ('Name', 'Error'): 638,
         ('Integer', 'Error'): 1